# Experiment 0527 - how good are GPT models in Fermi Estimation

In [ ]:
# dependency
%pip install python-dotenv
%pip install openai
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     --------------------------------------- 10.6/10.6 MB 14.9 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 10.5 MB/s eta 0:00:00
     ------------------------------------- 341.8/341.8 kB 10.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # load keys from .env file
import openai # use OpenAI API
import random
import json
import pandas as pd

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=300, # this is the maximum number of tokens to generate
    )
    print(">>>>RESPONSE>>>>")
    print(str(response.choices[0].message["content"]))
    print("<<<<END<<<<")
    return response.choices[0].message["content"]

In [ ]:
# Run this code block to get an example output
prompt_example= f"""What is Fermi Estimation and how does it work?"""
response = get_completion(prompt_example)

>>>>RESPONSE>>>>
Fermi estimation is a problem-solving technique that involves making rough estimates and approximations to arrive at a reasonable answer. It is named after the physicist Enrico Fermi, who was known for his ability to make quick and accurate estimates.

The basic idea behind Fermi estimation is to break down a complex problem into simpler parts and make educated guesses about the values of the variables involved. For example, if you were asked to estimate the number of people in a city, you might start by estimating the population of a few neighborhoods and then extrapolating that to the entire city.

To make a Fermi estimate, you typically follow these steps:

1. Identify the key variables involved in the problem.
2. Make educated guesses about the values of these variables.
3. Use these guesses to calculate an approximate answer.
4. Check your answer for reasonableness and adjust your estimates if necessary.

Fermi estimation is often used in science, engineering, and

## Prompt Design

In [ ]:
# prompt 1 - baseline (no context 0-shot)
def prompt_1(question):
    return f"""\
Here is a question: {question}. Give your reason and your estimated answer in the order of magnitute 10^x.
Reason:
Answer: x = ?
"""

In [ ]:
# prompt 2 - with classic example (rich context 1-shot)
def prompt_2(question):
    return f"""\
You are an AI assistant helping people practice Fermi estimation techniques.\
Fermi estimation is a problem-solving technique that involves making rough estimates and approximations to arrive at a reasonable answer.
The basic idea behind Fermi estimation is to break down a complex problem into simpler parts and make educated guesses about the values of the variables involved.
For example, if you were asked to estimate the number of people in a city, you might start by estimating the population of a few neighborhoods and then extrapolating that to the entire city.
To make a Fermi estimate, you typically follow these steps:
1. Identify the key variables involved in the problem.
2. Make educated guesses about the values of these variables.
3. Use these guesses to calculate an approximate answer.
4. Check your answer for reasonableness and adjust your estimates if necessary.

Here is a question: {question}. Give your reason and your estimated answer in the order of magnitute 10^x.
Reason:
Answer: x = ?
"""

In [ ]:
# prompt 3 - competence prompt: "you KNOW the answer in the order of magnitude" (context 0-shot)
def prompt_3(question):
    return f"""\
A curious learner is asking you a question:
{question}
You roughly know the answer in the order of magnitude, and you will teach the learner, step by step, how they can work out the answer themselves. Answer in the order of magnitude 10^x.
Reason:
Answer: x = ?
"""

In [ ]:
# prompt 4 - prompt with uncertainty "you DON'T KNOW the exact answer, but you know it's in the order of magnitude of 10^x" (context 0-shot)
def prompt_4(question):
    return f"""\
A curious learner is asking you a question:
{question}
You don't know the exact answer, but you will help the learner to work out a rough answer, step by step. Answer in the order of magnitude 10^x.
Reason:
Answer: x = ?
"""

In [ ]:
# prompt 5 - explicit step-by-step instruction (zero-shot)

# ask LLM to give its estimation chain of thought.
def prompt_5(question):
    return f"""\
You are an AI assistant helping people practice Fermi estimation techniques.\
A human will post a question and ask for help to estimate a quantity in the order of magnitude.\
The estimation should rely on human's common senses and knowledge about the world.

To make a Fermi estimate, you typically follow these steps as your reasons:
1. Identify the final quantity of interest, T, or the target quantity, that needs to be estimated.\
2. Analyse factors will determine the answer to the target quantity, and name those sub-quantities.\
3. Logically link relevant factors together to create a mathematical model for computation.
4. Present your final answer in the order of magnitude, in the form of 10^x, where x is the power of 10.

Question: {question}
Reason:
Answer: x = ?
"""

## Data collection / query the model

In [ ]:
# sample questions from Eric
# open json file
file_path = 'C:/Users/WTX/workspace/fermi-aid-tree/my-app/src/data/Eric_FP_suggestion.json'

def sample_questions(json_file):
    with open(json_file, 'r', encoding="utf-8") as f:
        data = json.load(f)

    questions = [entry['name'] for entry in data]
    sampled_questions = random.sample(questions, 20)
    return sampled_questions


sampled_questions = sample_questions(file_path)
for question in sampled_questions:
    print(question) 


How many hairs are on the average human head?
What is the annual consumption of ice cream, in pounds, for the State of Ohio?
How many toothpicks are required to cover a baseball diamond if the tooth picks are lying flat and sideby-side?
The number of Earths that would fit inside of the sun?
What is the maximum weight, in tons, of a Giant Sequoia tree?
If every person who ever lived crowded together in one spot, how much area would be covered, in terms of the area of Rhode Island?
If the annual interest that is owed on the national debt each year were divided equally amongst all U.S. citizens, how much would each citizen owe?
How many lawyers are there in the United States?
How many red blood cells are needed to blanket the surface area of an M&M?
The My Lai Massacre occurred on this date in 1968. According to the US army, how many people were killed?
How many car miles were avoided in 2010 by buses that took kids to school?
How many tons one cubic cm of a neutron star would be?
How man

### Experiment with test data

In [ ]:
# query using prompt_1 on GPT3.5
# write answers to a file for post analysis
# query using prompt_1 on GPT4
# write answers to a file for post analysis

# Define your DataFrame columns
columns = ['Question', 'Prompt_1_result', 'Prompt_2_result', 'Prompt_3_result', 'Prompt_4_result', 'Prompt_5_result']

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

counter = 0
prompt_list = [prompt_1, prompt_2, prompt_3, prompt_4, prompt_5]

for number, question in enumerate(sampled_questions[:10]):
    # print(f"Question {number+1}: {question}")
    # Create a dictionary to store the results for this question
    results = {'Question': question}
    
    for index, prompt in enumerate(prompt_list[:5]):
        # print(f"Prompt {index+1}")
        try:
            counter += 1
            # Assuming that get_completion function returns the answer
            # response = get_completion(prompt, model=model)
            # For the sake of this example, let's say the response is 'Answer'
            response = prompt(question)
            # Add the result to the results dictionary
            results[f'Prompt_{index+1}_result'] = response
        except Exception as e:
            print("===error message received===")
            print(e)
            print("NEXT QUESTION")
            # Add 'ERROR' to the results dictionary
            results[f'Prompt_{index+1}_result'] = 'ERROR'
    # At the end of the inner loop, append the results to the DataFrame
    df = pd.concat([df, pd.DataFrame([results])], ignore_index=True)
print(df)
df.to_csv('post_analysis.csv', index=False)

with open('output.txt', 'w') as f:
    for answer in df.values.flatten():
        f.write(str(answer) + '\n')
        f.write("========================================\n")

In [88]:
# looping through each model
# looping through each prompt
# looping through each question

sampled_questions_text = """How many hairs are on the average human head?
What is the annual consumption of ice cream, in pounds, for the State of Ohio?
How many toothpicks are required to cover a baseball diamond if the tooth picks are lying flat and sideby-side?
The number of Earths that would fit inside of the sun?
What is the maximum weight, in tons, of a Giant Sequoia tree?
If every person who ever lived crowded together in one spot, how much area would be covered, in terms of the area of Rhode Island?
If the annual interest that is owed on the national debt each year were divided equally amongst all U.S. citizens, how much would each citizen owe?
How many lawyers are there in the United States?
How many red blood cells are needed to blanket the surface area of an M&M?
The My Lai Massacre occurred on this date in 1968. According to the US army, how many people were killed?
How many car miles were avoided in 2010 by buses that took kids to school?
How many tons one cubic cm of a neutron star would be?
How many miles of railroad are operational in the U.S.?
Hurricane Sandy was devastating and expensive. If you had a pile of pennies equal in value to Sandy's cost, what would be its weight in troy ounces?
How many stacked 2 x 4 LEGO blocks would it take to reach the height of the Empire state Building?
How many steps does the average American walk during their lifetime?
According to the Census of Marine Life, with a membership of about 1000 scientists in 70 countries, what is the estimated number of life forms in the world's oceans?
How many active volcanoes are on the surface of the earth?
How many cups of coffee does Starbucks sell each year?
Determine the density of oxygen in the air of this room at 25°C and 760 torr in g/L."""
def string_to_list(input_string):
    return input_string.split('\n')


# prepare output in a csv file

# start querying
sampled_questions=string_to_list(sampled_questions_text)
# Define your DataFrame columns
columns = ['Question', 'Prompt_1_result', 'Prompt_2_result', 'Prompt_3_result', 'Prompt_4_result', 'Prompt_5_result']

# Create an empty DataFrame
df = pd.DataFrame(columns=columns)

counter = 0
prompt_list = [prompt_1, prompt_2, prompt_3, prompt_4, prompt_5]
model = 'gpt-3.5-turbo'
print(f"Let's ROCK! with {model}")
for number, question in enumerate(sampled_questions[:2]):
    # print(f"Question {number+1}: {question}")
    # Create a dictionary to store the results for this question
    results = {'Question': question}
    
    for index, prompt in enumerate(prompt_list[:5]):
        # print(f"Prompt {index+1}")
        try:
            counter += 1
            # Assuming that get_completion function returns the answer
            # response = get_completion(prompt, model=model)
            # Add the result to the results dictionary
            results[f'Prompt_{index+1}_result'] = response
        except Exception as e:
            print("===error message received===")
            print(e)
            print("NEXT QUESTION")
            # Add 'ERROR' to the results dictionary
            results[f'Prompt_{index+1}_result'] = 'ERROR'
    # At the end of the inner loop, append the results to the DataFrame
    df = pd.concat([df, pd.DataFrame([results])], ignore_index=True)

# console output
print(df)
# save data to csv for post analysis
df.to_csv('post_analysis.csv', index=False)
# save flattened data to txt file for easy copy-paste
with open('output.txt', 'w') as f:
    for answer in df.values.flatten():
        f.write(str(answer) + '\n')
        f.write("========================================\n")

Let's ROCK! with gpt-3.5-turbo
                                            Question   
0      How many hairs are on the average human head?  \
1  What is the annual consumption of ice cream, i...   

                                     Prompt_1_result   
0  You are an AI assistant helping people practic...  \
1  You are an AI assistant helping people practic...   

                                     Prompt_2_result   
0  You are an AI assistant helping people practic...  \
1  You are an AI assistant helping people practic...   

                                     Prompt_3_result   
0  You are an AI assistant helping people practic...  \
1  You are an AI assistant helping people practic...   

                                     Prompt_4_result   
0  You are an AI assistant helping people practic...  \
1  You are an AI assistant helping people practic...   

                                     Prompt_5_result  
0  You are an AI assistant helping people practic...  
1  You are an 